In [11]:
import psycopg2

In [52]:
from google.cloud import secretmanager

In [86]:
client = secretmanager.SecretManagerServiceClient()

In [87]:
project_id, secret_id = "e2e-pipeline-cs", "e2e-gcp-cloud-sql-secret"

In [88]:
secret_name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"

In [89]:
response = client.access_secret_version(name=secret_name)

In [90]:
import json

In [91]:
secret = json.loads(response.payload.data)

In [92]:
conn = psycopg2.connect(**secret)

In [93]:
curr = conn.cursor()

In [94]:
curr.execute("SELECT * FROM departments ORDER BY department_id LIMIT 5")

In [95]:
curr.fetchall()

[(2, 'Fitness'), (3, 'Footwear'), (4, 'Apparel'), (5, 'Golf'), (6, 'Outdoors')]

In [96]:
curr.execute(
    """
        SELECT order_status, COUNT(*) AS order_count
        FROM orders
        GROUP BY 1
        ORDER BY 2 DESC
    """
)

In [97]:
curr.fetchall()

[('COMPLETE', 22899),
 ('PENDING_PAYMENT', 15030),
 ('PROCESSING', 8275),
 ('PENDING', 7610),
 ('CLOSED', 7556),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558),
 ('CANCELED', 1428),
 ('PAYMENT_REVIEW', 729)]

In [31]:
import sqlalchemy as sa
import pandas as pd

In [38]:
engine = sa.create_engine(f"postgresql+psycopg2://{secret['user']}:{secret['password']}@{secret['host']}:{secret['port']}/{secret['database']}")

In [40]:
df = pd.read_sql_table("orders", con=engine)

In [41]:
df.head()

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25,11599,CLOSED
1,2,2013-07-25,256,PENDING_PAYMENT
2,3,2013-07-25,12111,COMPLETE
3,4,2013-07-25,8827,CLOSED
4,5,2013-07-25,11318,COMPLETE


In [46]:
daily_order_status_count = df \
                        .groupby(['order_date', 'order_status'])['order_customer_id'] \
                        .agg(order_count='count') \
                        .reset_index()

In [48]:
daily_order_status_count.to_sql("daily_order_status_count", con=engine, if_exists="replace", index=False)

203

In [50]:
pd.read_sql(
    """
        SELECT order_status, SUM(order_count) AS order_count
        FROM daily_order_status_count
        GROUP BY 1
        ORDER BY 2 DESC
    """,
    con=engine
)

,order_status,order_count
0,COMPLETE,22899.0
1,PENDING_PAYMENT,15030.0
2,PROCESSING,8275.0
3,PENDING,7610.0
4,CLOSED,7556.0
5,ON_HOLD,3798.0
6,SUSPECTED_FRAUD,1558.0
7,CANCELED,1428.0
8,PAYMENT_REVIEW,729.0
